# CP421 Project - Recommendation System(CF)
#### Shawn Davis
#### Adam Cassidy
#### Mengdan Wan 

##### Import rating and movie data

In [1]:
import pandas as pd
import numpy as np

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ml-latest-small/ml-latest-small/ratings.csv', skiprows=1, sep=',', header=None, names=rnames)

ratings[:]


,user_id,movie_id,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [2]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('ml-latest-small/ml-latest-small/movies.csv', skiprows=1, sep=',', header=None, names=mnames,)

movies

,movie_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


##### Merge movies and ratings

In [3]:
data = pd.merge(ratings, movies)
data

,user_id,movie_id,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller


##### Random split data for training and test (train = 0.9 and test = 0.1)

In [4]:
data_copy = data.copy()
train_set = data_copy.sample(frac=0.9) #random choose sample (change the frac to change the training ratio)
test_set = data_copy.drop(train_set.index)

train_set[:]

,user_id,movie_id,rating,timestamp,title,genres
65520,234,4519,4.0,1004409347,"Land Before Time, The (1988)",Adventure|Animation|Children|Fantasy
58697,372,1129,2.0,874416338,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
67460,274,56775,3.0,1239123021,National Treasure: Book of Secrets (2007),Action|Adventure
17328,610,68157,5.0,1493845023,Inglourious Basterds (2009),Action|Drama|War
14856,19,3033,2.0,965703811,Spaceballs (1987),Comedy|Sci-Fi
...,...,...,...,...,...,...
7179,153,1198,0.5,1525548264,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure
43937,137,1721,4.0,1204859407,Titanic (1997),Drama|Romance
30491,28,293,4.5,1234338073,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller
81989,489,1096,5.0,1332706013,Sophie's Choice (1982),Drama


In [5]:
train_mov = pd.DataFrame(train_set.loc[:,'title':'genres'])
train_no_dup = train_mov.drop_duplicates('title')
train_no_dup = train_no_dup.reset_index(drop=True)
train_no_dup

,title,genres
0,"Land Before Time, The (1988)",Adventure|Animation|Children|Fantasy
1,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
2,National Treasure: Book of Secrets (2007),Action|Adventure
3,Inglourious Basterds (2009),Action|Drama|War
4,Spaceballs (1987),Comedy|Sci-Fi
...,...,...
9359,"Escort, The (Scorta, La) (1993)",Crime|Thriller
9360,In the Bleak Midwinter (1995),Comedy|Drama
9361,Sharknado 3: Oh Hell No! (2015),Horror|Sci-Fi
9362,Carry On Don't Lose Your Head (1966),Comedy


In [6]:
GENRES = ["Action", "Adventure", "Animation" ,"Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
            "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western",
              "(no genres listed)"]

genres_matrix = []
for i in range(0, len(train_no_dup)):
    b = []
    mov = train_no_dup.loc[i, 'genres']
    #print(m)
    for g in GENRES:
        if g in mov:
            b.append(1)
        else:
            b.append(0)
    print(b)
    genres_matrix.append(b)
print(genres_matrix)

[0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0

[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 1

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0

In [7]:
gen = pd.DataFrame(genres_matrix, index = train_no_dup.loc[:, 'title'], columns = GENRES)
gen = gen.T
gen

title,"Land Before Time, The (1988)",Escape from New York (1981),National Treasure: Book of Secrets (2007),Inglourious Basterds (2009),Spaceballs (1987),Princess Mononoke (Mononoke-hime) (1997),Lost in Space (1998),Traffic (2000),Bridget Jones: The Edge of Reason (2004),Zodiac (2007),...,"Angels' Share, The (2012)",Morons From Outer Space (1985),California Split (1974),Presto (2008),Let's Get Harry (1986),"Escort, The (Scorta, La) (1993)",In the Bleak Midwinter (1995),Sharknado 3: Oh Hell No! (2015),Carry On Don't Lose Your Head (1966),Hereafter (2010)
Action,0,1,1,1,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Adventure,1,1,1,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Animation,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
Children,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
Comedy,0,0,0,0,1,0,0,0,1,0,...,1,1,1,1,0,0,1,0,1,0
Crime,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
Documentary,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Drama,0,0,0,1,0,1,0,1,1,1,...,1,0,1,0,0,0,1,0,0,1
Fantasy,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
Film-Noir,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
corr_gen = gen.corr(method = 'pearson')
corr_gen

title,"Land Before Time, The (1988)",Escape from New York (1981),National Treasure: Book of Secrets (2007),Inglourious Basterds (2009),Spaceballs (1987),Princess Mononoke (Mononoke-hime) (1997),Lost in Space (1998),Traffic (2000),Bridget Jones: The Edge of Reason (2004),Zodiac (2007),...,"Angels' Share, The (2012)",Morons From Outer Space (1985),California Split (1974),Presto (2008),Let's Get Harry (1986),"Escort, The (Scorta, La) (1993)",In the Bleak Midwinter (1995),Sharknado 3: Oh Hell No! (2015),Carry On Don't Lose Your Head (1966),Hereafter (2010)
title,,,,,,,,,,,,,,,,,,,,,
"Land Before Time, The (1988)",1.000000,0.050000,0.243544,-0.223607,-0.177123,0.570922,0.130437,-0.223607,-0.223607,-0.223607,...,-0.177123,-0.177123,-0.177123,0.683333,0.243544,-0.177123,-0.177123,-0.177123,-0.121716,0.243544
Escape from New York (1981),0.050000,1.000000,0.664211,0.130437,0.243544,0.277746,0.838525,0.130437,-0.223607,0.130437,...,-0.177123,0.243544,-0.177123,-0.266667,0.664211,0.243544,-0.177123,0.243544,-0.121716,-0.177123
National Treasure: Book of Secrets (2007),0.243544,0.664211,1.000000,0.321798,-0.117647,0.573944,0.792118,-0.148522,-0.148522,-0.148522,...,-0.117647,-0.117647,-0.117647,-0.177123,1.000000,-0.117647,-0.117647,-0.117647,-0.080845,-0.117647
Inglourious Basterds (2009),-0.223607,0.130437,0.321798,1.000000,-0.148522,0.396788,0.208333,0.208333,0.208333,0.208333,...,0.321798,-0.148522,0.321798,-0.223607,0.321798,-0.148522,0.321798,-0.148522,-0.102062,0.321798
Spaceballs (1987),-0.177123,0.243544,-0.117647,-0.148522,1.000000,-0.204980,0.321798,-0.148522,0.321798,-0.148522,...,0.441176,1.000000,0.441176,0.243544,-0.117647,-0.117647,0.441176,0.441176,0.687184,-0.117647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Escort, The (Scorta, La) (1993)",-0.177123,0.243544,-0.117647,-0.148522,-0.117647,-0.204980,-0.148522,0.792118,-0.148522,0.792118,...,-0.117647,-0.117647,-0.117647,-0.177123,-0.117647,1.000000,-0.117647,-0.117647,-0.080845,-0.117647
In the Bleak Midwinter (1995),-0.177123,-0.177123,-0.117647,0.321798,0.441176,0.184482,-0.148522,0.321798,0.792118,0.321798,...,1.000000,0.441176,1.000000,0.243544,-0.117647,-0.117647,1.000000,-0.117647,0.687184,0.441176
Sharknado 3: Oh Hell No! (2015),-0.177123,0.243544,-0.117647,-0.148522,0.441176,-0.204980,0.321798,-0.148522,-0.148522,-0.148522,...,-0.117647,0.441176,-0.117647,-0.177123,-0.117647,-0.117647,-0.117647,1.000000,-0.080845,-0.117647


In [9]:
train_set.describe()

,user_id,movie_id,rating,timestamp
count,90752.000000,90752.000000,90752.000000,9.075200e+04
mean,326.066423,19465.097849,3.500573,1.206023e+09
std,182.646059,35546.965998,1.041925,2.162179e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1200.000000,3.000000,1.018291e+09
50%,325.000000,2990.000000,3.500000,1.186087e+09
75%,477.000000,8169.000000,4.000000,1.435995e+09
max,610.000000,193587.000000,5.000000,1.537799e+09


##### Get mean rating of movies

In [10]:
ratings = pd.DataFrame(train_set.groupby('title')['rating'].mean())
ratings

,rating
title,
'71 (2014),4.000000
'Hellboy': The Seeds of Creation (2004),4.000000
'Round Midnight (1986),3.500000
'Salem's Lot (2004),5.000000
'Til There Was You (1997),5.000000
...,...
eXistenZ (1999),3.850000
xXx (2002),2.681818
xXx: State of the Union (2005),2.000000


##### Count how many ratings of each movie

In [11]:
ratings['number_of_ratings'] = data.groupby('title')['rating'].count()
ratings.sort_values(by='number_of_ratings', ascending=False).head(10)

,rating,number_of_ratings
title,,
Forrest Gump (1994),4.164360,329
"Shawshank Redemption, The (1994)",4.413495,317
Pulp Fiction (1994),4.183019,307
"Silence of the Lambs, The (1991)",4.140562,279
"Matrix, The (1999)",4.170782,278
Star Wars: Episode IV - A New Hope (1977),4.233333,251
Jurassic Park (1993),3.723256,238
Braveheart (1995),3.973684,237
Terminator 2: Judgment Day (1991),3.930412,224


In [12]:
#import seaborn as sns
#sns.jointplot(x='rating', y='number_of_ratings', data=ratings)

In [13]:
train_matrix = pd.pivot_table(train_set, values='rating', index='user_id', columns='title')
#train_matrix.fillna(0, inplace=True)
train_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


##### Get pearson correlation matrix of movies

In [14]:
corr_matrix = train_matrix.corr(method = 'pearson')
corr_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),NaN,NaN,NaN,NaN,NaN,NaN,0.188982,NaN,-0.388883,1.0,...,NaN,NaN,NaN,NaN,NaN,1.000000,0.762493,NaN,-1.0,NaN
xXx (2002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.823239,NaN,...,NaN,-0.086066,-1.0,NaN,NaN,0.762493,1.000000,0.94388,0.5,NaN
xXx: State of the Union (2005),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,...,NaN,-0.755929,NaN,NaN,NaN,NaN,0.943880,1.00000,NaN,NaN


#### Top 10 recommendation movies which every movies are rated at least N times by users.

In [20]:
N = 20
#some movies only has negtive similarities with other movies, set threshold to -1 to see the result
similarity_Threshold = 0    
movie_i = input("Enter the movie name: ")

if (corr_matrix.index == movie_i).any():   #recommend by collaborative filtering
    recom = pd.DataFrame(corr_matrix[movie_i])
    recom.dropna(inplace=True)
    recom = recom.join(ratings['number_of_ratings'])
    recom = recom[recom['number_of_ratings'] > N]
    #recom = recom[recom[movie_i] < 1] 
    recom = recom[recom[movie_i] > similarity_Threshold] 
    recom_n = recom.sort_values(by=movie_i, ascending=False).head(10)
    print(recom_n.empty)
    print("1")
    if recom_n.empty:   #recommend by content filtering using genres
        recom = pd.DataFrame(corr_gen[movie_i])
        recom.dropna(inplace=True)
        recom = recom.join(ratings['number_of_ratings'])
        recom = recom[recom['number_of_ratings'] > N]
        #recom = recom[recom[movie_i] < 1] 
        recom = recom[recom[movie_i] > similarity_Threshold] 
        recom_n = recom.sort_values(by=movie_i, ascending=False).head(10)
        print(recom_n.empty)
        print("2")
else:
    if (corr_gen.index == movie_i).any():   #recommend by content filtering using genres
        recom = pd.DataFrame(corr_gen[movie_i])
        recom.dropna(inplace=True)
        recom = recom.join(ratings['number_of_ratings'])
        recom = recom[recom['number_of_ratings'] > N]
        #recom = recom[recom[movie_i] < 1] 
        recom = recom[recom[movie_i] > similarity_Threshold]  
        recom_n = recom.sort_values(by=movie_i, ascending=False).head(10)
        print(recom_n.empty)
        print("3")
        
    else:    #recommend by content filtering using genres, for new movies.
        new_g = []
        b = []
        for genre in GENRES:
            if genre in g:
                b.append(1)
            else:
                b.append(0)
        new_g.append(b)
        current = pd.DataFrame(new_g, index=[movie_i],columns=GENRES)
        current = current.T

        gen = gen.join(current)

        corr_gen = gen.corr(method = 'pearson')
        recom = pd.DataFrame(corr_gen[movie_i])
        recom.dropna(inplace=True)
        recom = recom.join(ratings['number_of_ratings'])
        recom = recom[recom['number_of_ratings'] > N]
        #recom = recom[recom[movie_i] < 1] 
        recom = recom[recom[movie_i] > similarity_Threshold] 
        recom_n = recom.sort_values(by=movie_i, ascending=False).head(10)
        print(recom_n.empty)
        print("4")
        
recom_n



Enter the movie name: 'Salem's Lot (2004)
True
1
False
2


,'Salem's Lot (2004),number_of_ratings
"Others, The (2001)",1.000000,55.0
Copycat (1995),0.864099,36.0
"Sixth Sense, The (1999)",0.838525,179.0
What Lies Beneath (2000),0.838525,25.0
"Blair Witch Project, The (1999)",0.838525,64.0
Rosemary's Baby (1968),0.838525,32.0
I Know What You Did Last Summer (1997),0.838525,32.0
"Cell, The (2000)",0.838525,45.0
"Ring, The (2002)",0.838525,47.0
Saw (2004),0.838525,33.0


##### Read test data and convert it to numpy array

In [21]:
test_set.drop(['movie_id', 'timestamp'], axis=1,inplace=True)
test_set

,user_id,rating,title,genres
7,21,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
8,27,3.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
11,33,3.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
12,40,5.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
13,43,5.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...
100810,610,3.5,The Green Inferno (2014),Horror|Thriller
100820,610,3.5,Cooties (2015),Comedy|Horror
100829,610,3.5,Southbound (2016),Horror
100831,610,2.5,Bloodmoon (1997),Action|Thriller


In [22]:
print(len(test_set))
test_array = test_set.to_numpy()
test_array

10084


array([[21, 3.5, 'Toy Story (1995)',
        'Adventure|Animation|Children|Comedy|Fantasy'],
       [27, 3.0, 'Toy Story (1995)',
        'Adventure|Animation|Children|Comedy|Fantasy'],
       [33, 3.0, 'Toy Story (1995)',
        'Adventure|Animation|Children|Comedy|Fantasy'],
       ...,
       [610, 3.5, 'Southbound (2016)', 'Horror'],
       [610, 2.5, 'Bloodmoon (1997)', 'Action|Thriller'],
       [610, 4.5, 'Sympathy for the Underdog (1971)',
        'Action|Crime|Drama']], dtype=object)

##### Go through every test data, and using pearson correlation matrix to find top k similar movies that already rated by current user. 
##### If user did not rate any similar movie, then use the average rating as predicting of this movie
##### If no similar movie or the movie did not in trainning data(New movies), then using 'Content Filtering' by genres to find the similar movies, and make predicting by using average rating of the top 20 similar movies which filtering by genres.

In [23]:
pred = []
k = 20
c=0
noOfRatingThreshold = 20

for m in test_array:
    t = m[2]
    u = m[0]
    g = m[3]
    if (corr_matrix.index == t).any():
        movie_corr = pd.DataFrame(corr_matrix[t])
        movie_corr.dropna(inplace=True)
        #movie_corr = movie_corr.join(ratings['number_of_ratings'])
        #movie_corr = movie_corr[movie_corr['number_of_ratings'] > noOfRatingThreshold]
        movie_corr = movie_corr[movie_corr[t] < 1] 
        movie_corr = movie_corr[movie_corr[t] > 0] 
        sim_movies = movie_corr.sort_values(by=t, ascending=False)
        #sim_movies
        sim_title = sim_movies.index
        #print(sim_title)

        count = 0
        r_s = 0
        total_sim = 0
        for movie in sim_title:
            r = train_matrix.loc[u, movie]
            #print(u)
            #print(movie)
            if not np.isnan(r):
                #print(r)
                count += 1
                sim = sim_movies.loc[movie, t]
                r_s += r*sim
                total_sim += sim
            if count >= k:
                break
                #print(count)
        
        if total_sim == 0:
            
            g_corr = pd.DataFrame(corr_gen[t])
            g_corr.dropna(inplace=True)
            #movie_corr = movie_corr.join(ratings['number_of_ratings'])
            #movie_corr = movie_corr[movie_corr['number_of_ratings'] > noOfRatingThreshold]
            g_corr = g_corr[g_corr[t] < 1] 
            g_corr = g_corr[g_corr[t] > 0] 
            sim_gen = g_corr.sort_values(by=t, ascending=False)
            sim_genres = sim_gen.index
            #print(sim_title)
            
            count_1 = 0
            r_s_1 = 0
            total_sim_1 = 0
            for tit in sim_genres:
                if (corr_matrix.index == tit).any():
                    rt = train_matrix.loc[u, tit]
                    #print(u)
                    #print(movie)
                    if not np.isnan(rt):
                        count_1 += 1
                        sim1 = sim_gen.loc[tit, t]
                        #print(sim)
                        r_s_1 += rt*sim1
                        total_sim_1 += sim1
                    if count_1 >= k:
                        break
                        #print(count)

            if total_sim_1 == 0:
                rate = ratings.loc[tit, 'rating']
                
            else:
                rate = r_s_1/total_sim_1
                
            pred.append(rate)
            print(rate)
            c += 1
            
        else:
            rate = r_s/total_sim
            pred.append(rate)
            print(rate)
            c += 1
    else:
        if (corr_gen.index == t).any():
            curr_corr = pd.DataFrame(corr_gen[t])
            curr_corr.dropna(inplace=True)
            #movie_corr = movie_corr.join(ratings['number_of_ratings'])
            #movie_corr = movie_corr[movie_corr['number_of_ratings'] > noOfRatingThreshold]
            curr_corr = curr_corr[curr_corr[t] < 1] 
            curr_corr = curr_corr[curr_corr[t] > 0] 
            sim_gen = curr_corr.sort_values(by=t, ascending=False)
            #sim_movies
            sim_tit = sim_gen.index
        else:
            #print("new")
            g_a = []
            b = []
            for genre in GENRES:
                if genre in g:
                    b.append(1)
                else:
                    b.append(0)
            #print(b)
            g_a.append(b)
            current = pd.DataFrame(g_a, index=[t],columns=GENRES)
            current = current.T
            current

            gen = gen.join(current)
            #gen
            corr_gen = gen.corr(method = 'pearson')
            #corr_gen
            curr_corr = pd.DataFrame(corr_gen[t])
            curr_corr.dropna(inplace=True)
            #movie_corr = movie_corr.join(ratings['number_of_ratings'])
            #movie_corr = movie_corr[movie_corr['number_of_ratings'] > noOfRatingThreshold]
            curr_corr = curr_corr[curr_corr[t] < 1] 
            curr_corr = curr_corr[curr_corr[t] > 0] 
            sim_gen = curr_corr.sort_values(by=t, ascending=False)
            #sim_movies
            sim_tit = sim_gen.index

        count_2 = 0
        r_s_2 = 0
        tot_r = 0

        for tit in sim_tit:
            if (ratings.index == tit).any():
                rt2 = ratings.loc[tit, 'rating']
                count_2 += 1
                tot_r += rt2
            if count_2 >= 20:
                break
                
        
        if count_2 == 0:    #No genres, set the rate to 0
            rate = 0
        else:
            rate = tot_r/count_2
        print(rate)
        pred.append(rate)
        
        #print("No Recommendation(New movie)")
        c += 1
    print(c)
print("Total movies: %d" % c)
print(len(pred))



3.201979777443073
1
3.6397713715214977
2
3.637063235623603
3
3.8082223597711122
4
4.656183413918059
5
4.172720824084027
6
2.865311902838126
7
3.3207927716792818
8
4.268702806697744
9
3.047519476942948
10
4.1743687630164
11
4.3515780747085895
12
3.187485954723254
13
3.0997888272883274
14
3.986756161469958
15
2.8455032055273346
16
2.741733037669522
17
3.4400846966124394
18
3.320594959257635
19
3.1267915941838544
20
3.8252101576614352
21
3.205076435772492
22
3.956828241983348
23
3.601410657895661
24
4.27318807562382
25
3.833317174179563
26
4.028969204846476
27
3.5934830173222503
28
3.621531055925669
29
3.6748288484938976
30
3.569279372199484
31
3.6622415785772446
32
2.821189679977059
33
3.094683828025432
34
3.7312209008296278
35
3.6142924314599374
36
3.6763054971453863
37
3.566404899187443
38
4.375188841480748
39
3.595398673166106
40
3.214857935027914
41
2.877326223100466
42
3.9728459405482033
43
3.366209544828924
44
4.488680091769699
45
4.0060506649109024
46
3.587392993266355
47
3.585719

3.8790434140218633
377
3.989758833924621
378
3.37893502309309
379
3.977502190744163
380
3.766892540666453
381
2.941345149162465
382
2.8616875890055034
383
3.6763600862809938
384
3.547057825147048
385
3.221063287501139
386
3.619409502385453
387
3.5801218217910415
388
3.7696687763597554
389
3.0341950636690287
390
3.5362614987965286
391
2.34340971047188
392
2.601607836264926
393
3.2451103324646264
394
3.2370485261561743
395
3.2153629116045566
396
4.119349586600524
397
3.4827893291919487
398
3.072547780557173
399
3.658189382900002
400
3.9239708778342193
401
3.8028244706883942
402
2.9276606840818626
403
3.592233544849131
404
3.3183432712712686
405
2.8671453702121132
406
4.084176882803336
407
4.244736612919685
408
4.886461772559134
409
3.5128368441555518
410
3.9184760118427904
411
3.862960842904411
412
3.945936586031536
413
4.129817228221611
414
3.9956469552513623
415
3.0516721559801394
416
4.022350686316817
417
3.38578128478037
418
3.824601879155238
419
3.0369095399233847
420
4.058783091616

4.353807132913035
750
3.2577596393667956
751
3.739148511264308
752
3.4555367207297096
753
3.496432509681363
754
3.2338313149320665
755
3.9259124688545874
756
3.513906275502888
757
3.058234617360785
758
3.2499600600306433
759
3.8009191452054667
760
2.5962141463573345
761
2.5927111079144973
762
3.174619260582942
763
3.9192808677055555
764
3.8510010006013684
765
2.4205299169205974
766
3.0518282900525304
767
3.166166489453487
768
4.1322973323590455
769
4.552823818088025
770
3.2932914988073874
771
3.5952237242185157
772
3.558599708886966
773
2.886230601454177
774
3.5872254013656764
775
4.322028418732998
776
4.011062954842551
777
3.2580141469932444
778
4.131988880335744
779
2.525293593331788
780
3.7154145013197923
781
2.8664147525112478
782
4.034697956878153
783
4.382083473055286
784
4.239280529605324
785
3.361511704250835
786
4.439767136005222
787
3.494671480085928
788
3.119718427196901
789
3.660423964311299
790
4.69775832578893
791
4.156268678956119
792
3.109288704737576
793
3.539096723702

3.7751691700076377
1131
2.781683224559654
1132
2.9473933886677846
1133
3.4428190771413365
1134
3.528767879287873
1135
3.549852508426696
1136
3.8488352377864197
1137
2.8958317866571144
1138
3.501115001483302
1139
4.221913354616312
1140
3.5483833148674875
1141
3.269578118537016
1142
3.6500766429716163
1143
2.6447210234751632
1144
3.2711352276121524
1145
3.88520795552921
1146
3.5266783541527373
1147
3.0500319995337457
1148
3.3426363964719106
1149
3.4085243960346348
1150
4.194389853934634
1151
2.641755027129415
1152
3.600051925169971
1153
3.2102666835620473
1154
3.478922387920174
1155
2.390787750913979
1156
3.859065853903881
1157
3.3858349628175013
1158
2.5972482517815987
1159
3.2868025464983446
1160
2.978731958539112
1161
3.4054208621892013
1162
2.6555885024540284
1163
3.6992463508926363
1164
2.737414605114336
1165
4.164477629216794
1166
3.4511073733108413
1167
3.6009668908294565
1168
2.9460650629917997
1169
3.425139281255908
1170
4.219061213945214
1171
3.0565525490432077
1172
2.754090866

4.288923115854248
1486
3.622641141016259
1487
3.6776144961620796
1488
3.189559911164474
1489
3.7466129590153647
1490
3.776857741812417
1491
3.3702168866547564
1492
3.0568202663955275
1493
3.6661589589604313
1494
2.9233748509962028
1495
3.0552233254267445
1496
3.755801379705982
1497
3.8264653454096855
1498
3.1250698764154103
1499
2.8495224616933394
1500
3.2504072547069116
1501
2.9976919942763023
1502
3.6775902807908447
1503
2.953470332905756
1504
4.048816501165265
1505
4.499183853233382
1506
3.4695647478151215
1507
4.021255102672356
1508
3.109754391919296
1509
3.5445263094969257
1510
3.2675260583533663
1511
3.0320027873378432
1512
2.2806024441699853
1513
2.3334681446671484
1514
3.5976415458469417
1515
3.1999339740263917
1516
4.012331480697551
1517
2.8604318635368533
1518
4.251482754050833
1519
4.003176807955547
1520
3.9377155763791984
1521
3.2125347683509395
1522
3.792270684310949
1523
3.7662604322106366
1524
4.5671275000571665
1525
4.1957756415812595
1526
4.000730033082431
1527
3.90604

...

3.771350732213422
8634
3.1563835293344225
8635
4.472913158831619
8636
3.623400656413929
8637
3.500000000000001
8638
3.735930570953851
8639
4.202383818344443
8640
4.35294495689794
8641
4.105335149196736
8642
3.954745374768507
8643
3.8004693384873702
8644
4.078152896056539
8645
3.426709868440733
8646
4.189657721875977
8647
3.800864894572602
8648
3.504996964008614
8649
2.547344443421027
8650
3.216211465961247
8651
4.070133795669804
8652
3.2745595331463178
8653
3.3573894625066028
8654
3.788054283222822
8655
3.1192198642110944
8656
2.5506861678303063
8657
4.6
8658
2.9224948822699837
8659
3.8419998241734183
8660
3.5042053271723317
8661
3.9926096901082517
8662
3.272738713163522
8663
3.745304817731289
8664
2.9796057513914653
8665
3.897771910906021
8666
3.3752668416118325
8667
2.9796057513914653
8668
3.4635674027277945
8669
3.5663082708573626
8670
3.050232781302283
8671
3.297642568086112
8672
3.9951042267375225
8673
3.0603516000126363
8674
4.029128868090532
8675
3.4752325595761198
8676
2.806543

3.4193154761904765
8993
4.154752255166156
8994
4.496173190593796
8995
3.409698060859818
8996
4.474511074896439
8997
3.8237535442470842
8998
3.0740081107321444
8999
3.1748593600059074
9000
3.833890544228247
9001
3.4972463239788785
9002
4.457395773530593
9003
4.01805999674921
9004
3.7807142857142857
9005
3.9909773800803343
9006
3.8708393285058444
9007
2.9746585971791943
9008
3.2666666666666666
9009
4.40909090909091
9010
4.222222222222222
9011
4.038442579162844
9012
3.6396602469627064
9013
3.524999999999999
9014
2.1999999999999993
9015
3.9886341993308454
9016
3.4265974247994953
9017
3.724999999999999
9018
2.699999999999998
9019
3.4362179487179487
9020
3.629718688858299
9021
4.0442698313399985
9022
4.541284611577301
9023
4.001204777633459
9024
2.3999999999999995
9025
4.40310195531377
9026
4.354420495585041
9027
3.3032125768967875
9028
3.875000000000001
9029
3.9724832598609945
9030
3.677766426816023
9031
3.9777033855523016
9032
3.290681348235774
9033
3.681811964527352
9034
3.454837998384729

3.7500000000000004
9349
4.04229505186373
9350
3.75
9351
4.0
9352
4.467484474355363
9353
3.764410589553673
9354
4.527589336090558
9355
4.150226496530502
9356
4.077032948798757
9357
4.100872159360054
9358
3.3800776431437756
9359
3.5
9360
2.906688568323794
9361
4.687526075435812
9362
3.513190093811907
9363
3.6250000000000013
9364
2.7429840307388207
9365
3.970647531103982
9366
3.493110576677235
9367
3.0271184554535155
9368
2.672777777777778
9369
4.811197596754911
9370
3.525000000000001
9371
4.22318861757485
9372
2.750000000000001
9373
3.159098947920585
9374
5.0
9375
2.8571148991859228
9376
1.8000000000000007
9377
3.725000000000001
9378
2.8557740276052628
9379
3.174789581671283
9380
3.7471872762080047
9381
3.9788695025411056
9382
2.723471646276139
9383
3.95
9384
4.217042123851468
9385
3.045575396825397
9386
2.1500000000000004
9387
3.305946739147645
9388
3.7783938164836433
9389
3.8977664778360945
9390
3.4302146082574483
9391
3.2027015124134586
9392
2.7086150248958876
9393
2.9241957831259056


3.377040488177917
9710
3.2904247835497835
9711
3.2904247835497835
9712
3.6900282243736697
9713
3.303200973391913
9714
4.150576796339114
9715
4.833333333333334
9716
3.954512130947038
9717
0
9718
3.461616606390385
9719
3.4840869836182335
9720
3.8057142857142856
9721
3.067752525252525
9722
4.331616615419005
9723
3.4445299615259293
9724
3.475000000000001
9725
3.4750000000000005
9726
3.5177486924990395
9727
3.659847563603812
9728
3.5340584447377923
9729
3.1750000000000003
9730
3.8803200973391916
9731
3.240708874458874
9732
3.623911071801931
9733
3.875
9734
3.5731921539504152
9735
2.6839909827535817
9736
3.43125
9737
3.760448340314575
9738
2.8750000000000004
9739
3.0342068158639415
9740
2.6750000000000007
9741
3.416544480139138
9742
2.60252776311113
9743
3.16639785426253
9744
3.178764021090095
9745
3.43324358974359
9746
3.5469422086173803
9747
3.280054866063817
9748
3.224999999999999
9749
3.824464285714286
9750
3.7323794106739614
9751
2.6874110395712645
9752
3.275
9753
3.2271722822714075
975

2.8605120235571
10069
3.3397954297932726
10070
3.0349179292929294
10071
3.0979322863897325
10072
2.6669769813519815
10073
3.2198973575444163
10074
2.776996468443837
10075
2.973008241758242
10076
3.1449015057084213
10077
3.287877886002886
10078
3.2679398576007896
10079
2.729372323642101
10080
3.2424467307453027
10081
2.6671456216739236
10082
2.93754329296248
10083
3.527005524462923
10084
Total movies: 10084
10084


#### RMSE measure (for k = 20, choose top 20 similar movies)

In [27]:
from sklearn.metrics import mean_squared_error
from math import sqrt

actuals = []
for i in range(0, len(test_array)):
    actuals.append(test_array[i][1])
#print(actuals)
rmse = sqrt(mean_squared_error(actuals, pred))
print("RMSE: %f" % rmse)

RMSE: 0.962702
